# Recommender System with Python and SQLite

In [207]:
import sqlite3
import numpy as np
import pandas as pd
import scipy
import implicit
import random
from datetime import datetime

pd.set_option('display.max_rows', 15)
pd.set_option('display.max_columns', 10)

### 1. Data cleaning and exploration

In [2]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx'

data = pd.read_excel(url)

data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
data.info()

# There are null values in CustomerID. We need to know who made the purchase to build for collaborative filtering

In [3]:
data = data[pd.notna(data['CustomerID'])]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      406829 non-null object
StockCode      406829 non-null object
Description    406829 non-null object
Quantity       406829 non-null int64
InvoiceDate    406829 non-null datetime64[ns]
UnitPrice      406829 non-null float64
CustomerID     406829 non-null float64
Country        406829 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.9+ MB


In [4]:
items = data.groupby(['StockCode','Description'], as_index=False).agg({'Quantity': np.sum})
items = items.sort_values(by = 'Quantity', ascending = False)
items.head()

,StockCode,Description,Quantity
2712,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,53215
3586,85099B,JUMBO BAG RED RETROSPOT,45066
2818,84879,ASSORTED COLOUR BIRD ORNAMENT,35314
3593,85123A,WHITE HANGING HEART T-LIGHT HOLDER,34147
361,21212,PACK OF 72 RETROSPOT CAKE CASES,33409


In [5]:
items.tail()

# More abnormalities in the data. We need to remove the items with negative quantity

,StockCode,Description,Quantity
314,21144,PINK POODLE HANGING DECORATION,-12
3910,CRUK,CRUK Commission,-16
629,21645,ASSORTED TUTTI FRUTTI ROUND BOX,-24
3911,D,Discount,-1194
2723,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-1460


In [6]:
data = data[data['Quantity'] > 0]
data['CustomerID'] = pd.to_numeric(data['CustomerID'], downcast = 'integer')

# Remove administrative charges such as postage or carriage

data['StockCodeLen'] = data['StockCode'].map(str).apply(len)
data = data[data['StockCodeLen'] > 4].drop('StockCodeLen', axis = 1)
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


### 2. Calculating sparsity

In order for collaborative filtering to work, we need to make sure the sparsity level does not go over 99.5%

In [7]:
# The score will be based on how many times a customer has purchased an item converted to 1-5 scale

recData = data.groupby(['CustomerID','StockCode','Description'], as_index = False).agg({'InvoiceNo': 'nunique'})

recData.rename(columns = {'InvoiceNo':'score'}, inplace = True)

In [8]:
pivotData = pd.pivot_table(recData, values = 'score', index = 'CustomerID', 
                            columns = 'Description', aggfunc = np.sum)
masked = np.ma.masked_invalid(pivotData)
mc = masked.compressed()
print(1 - mc.shape[0]/np.prod(pivotData.shape))

# The maximum recommended sparsity is 99.5%

0.9841981993871773


### 3. Collaborative filtering

In [9]:
# Create a sparse matrix

recData['CustomerID'] = recData['CustomerID'].astype("category")
recData['StockCode'] = recData['StockCode'].astype("category")
recData['user'] = recData['CustomerID'].cat.codes
recData['item'] = recData['StockCode'].cat.codes

# The implicit library expects data as a item-user matrix so we
# create two matricies, one for fitting the model (item-user) 
# and one for recommendations (user-item)
sparseItemUser = scipy.sparse.csr_matrix((rec_data['score'].astype(float), (rec_data['item'], rec_data['user'])))
sparseUserItem = scipy.sparse.csr_matrix((rec_data['score'].astype(float), (rec_data['user'], rec_data['item'])))

# Initialize the als model and fit it using the sparse item-user matrix
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=50)

# Calculate the confidence by multiplying it by alpha value, based on the methodologies of 
# http://yifanhu.net/PUB/cf.pdf
alphaVal = 15
dataConf = (sparseItemUser * alphaVal).astype('double')

#Fit the model
model.fit(data_conf)


100%|██████████| 50.0/50 [00:04<00:00, 11.06it/s]


In [80]:
# Helper functions

def getUserItems(user):
    """
    returns a list of item descriptions a user bought
    
    user: user categorical code 
    """
    return recData[recData['user'] == user]['Description'].astype('str').unique()

def getItemDescription(items):
    """
    returns a list of descriptions given a list of items
    
    items: list of item categorical code
    """
    descriptions = []
    for item in items:
        descriptions.extend(recData[recData['item'] == item ]['Description'].astype('str').unique())
    return descriptions

def getUserRecommendations(user):
    """
    returns a dictionary of item categorical code and score
    
    user: user categorical code
    """
    itemRecs = {}
    recommended = model.recommend(user, sparseUserItem)
    for item in recommended:
        itemRecs[item[0]] = item[1]
    return itemRecs

def getSimilarItems(item):
    """
    returns a dictionary of item categorical code and score
    
    item: item categorical code
    """
    similar = model.similar_items(item, 11)
    similar.pop(0)
    similarItems = {}
    for item in similar:
        similarItems[item[0]] = item[1]
    return similarItems

In [83]:
# Check the recommender system
print("User purchased:", getUserItems(6))

print("Recomendations:", getItemDescription(getUserRecommendations(6).keys()))

print("User purchased:", getUserItems(3000))

print("Recomendations:", getItemDescription(getUserRecommendations(3000).keys()))

# User 6 bought mainly cake related items and we were able to recommend more items related to baking
# Same with user 300, the user bought many lunch bags and the recommender outputted more suggestions for lunch bags

User purchased: ['NOVELTY BISCUITS CAKE STAND 3 TIER' 'MINI CAKE STAND WITH HANGING CAKES'
 'CERAMIC CAKE STAND + HANGING CAKES' 'CERAMIC CAKE BOWL + HANGING CAKES']
Recomendations: ['MINI CAKE STAND  HANGING STRAWBERY', 'LARGE CAKE STAND  HANGING STRAWBERY', 'STRAWBERRY FAIRY CAKE TEAPOT', 'SWEETHEART CERAMIC TRINKET BOX', 'CERAMIC BOWL WITH STRAWBERRY DESIGN', 'CERAMIC CAKE DESIGN SPOTTED MUG', 'STRAWBERRY CERAMIC TRINKET POT', 'STRAWBERRY CERAMIC TRINKET BOX', 'CERAMIC STRAWBERRY CAKE MONEY BANK', 'CERAMIC STRAWBERRY DESIGN MUG', 'CERAMIC CAKE DESIGN SPOTTED PLATE']
User purchased: ['STRAWBERRY CERAMIC TRINKET BOX' 'JUMBO STORAGE BAG SUKI'
 'LUNCH BAG SPACEBOY DESIGN ' 'LUNCH BAG SUKI  DESIGN '
 'JUMBO BAG SPACEBOY DESIGN' 'PLASTERS IN TIN SPACEBOY'
 'SPACEBOY LUNCH BOX ' 'CERAMIC STRAWBERRY CAKE MONEY BANK'
 'FAIRY CAKES NOTEBOOK A6 SIZE' 'JUMBO BAG RED RETROSPOT']
Recomendations: ['RED RETROSPOT CHARLOTTE BAG', 'LUNCH BAG RED RETROSPOT', 'LUNCH BAG RED SPOTTY', 'LUNCH BAG CARS BLU

In [84]:
# Check similar item recommendation
print("Original item:", getItemDescription([6]))

print("Recomendations:", getItemDescription(getSimilarItems(6).keys()))

print("Original item:", getItemDescription([300]))

print("Recomendations:", getItemDescription(getSimilarItems(300).keys()))

# The essense of the items are captured. The first one is a toy pen and the recommender outputted toys and other pens 
# The second item is a celebratory candle and all the recommendations are candles and party supplies

Original item: ['ASSTD DESIGN RACING CAR PEN']
Recomendations: ['CAMOUFLAGE LED TORCH', '3D DOG PICTURE PLAYING CARDS', '3D SHEET OF DOG STICKERS', 'NINJA RABBIT BLACK', 'BALLOON WATER BOMB PACK OF 35', 'NINJA RABBIT PINK', 'COLUMBIAN CANDLE RECTANGLE', 'CAMOUFLAGE EAR MUFF HEADPHONES', 'SILVER DIAMANTE PEN IN GIFT BOX', 'EIGHT PIECE SNAKE  SET']
Original item: ['SET OF 6 GIRLS CELEBRATION CANDLES']
Recomendations: ['SET 6 FOOTBALL CELEBRATION CANDLES', 'SET 20 NAPKINS FAIRY CAKES DESIGN ', 'DENIM PATCH PURSE PINK BUTTERFLY', 'PINK PARTY BAGS', 'SET/10 PINK POLKADOT PARTY CANDLES', 'PARTY INVITES JAZZ HEARTS', 'ALPHABET HEARTS STICKER SHEET', 'SET/10 IVORY POLKADOT PARTY CANDLES', 'HEARTS  STICKERS', 'SET/10 BLUE POLKADOT PARTY CANDLES']


### 4. Import recommendation data to SQLight

In [117]:
uniqueUser = recData[['user', 'CustomerID']].drop_duplicates()
uniqueItems = recData[['item', 'StockCode']].drop_duplicates()

In [227]:
# I am assuming original data is already in SQLite
con = sqlite3.connect("Database/recommender.sqlite")
cur = con.cursor()
cur.execute("DROP TABLE IF EXISTS user_recommendation;")
cur.execute("DROP TABLE IF EXISTS similar_items;")
cur.execute("CREATE TABLE user_recommendation (CustomerID INTEGER, StockCode TEXT, Score TEXT);")
cur.execute("CREATE TABLE similar_items (StockCode TEXT, SimilarStockCode TEXT, Score TEXT);")
cur.execute("CREATE TABLE IF NOT EXISTS import_status (created_at VARCHAR, name VARCHAR, status TEXT);")

In [228]:
# Insert customer recommendation

for row in uniqueUser.iterrows():
    rec = getUserRecommendations(row[1][0])
    for item, score in rec.items():
        cur.execute("INSERT INTO user_recommendation VALUES(?,?,?)", 
                    (int(row[1][1]), 
                     uniqueItems['StockCode'].loc[uniqueItems['item'] == item].values.astype(str)[0], float(score)))
        


In [229]:
# Insert item recommendation

for row in uniqueItems.iterrows():
    rec = getSimilarItems(row[1][0])
    for item, score in rec.items():
        cur.execute("INSERT INTO similar_items VALUES(?,?,?)", 
                    (str(row[1][0]), 
                     uniqueItems['StockCode'].loc[uniqueItems['item'] == item].values.astype(str)[0], float(score)))

con.commit()

In [245]:
# Verify whether the insertions went through by comparing number of rows

cur.execute("SELECT COUNT(DISTINCT CustomerID) FROM user_recommendation")
if cur.fetchone()[0] == len(uniqueUser):
    cur.execute("INSERT INTO import_status VALUES(?,?,?)", (str(datetime.now()), "user_recommendation","success"))
else:
    cur.execute("INSERT INTO import_status VALUES(?,?,?)", (str(datetime.now()), "user_recommendation","fail"))
                
cur.execute("SELECT COUNT(DISTINCT StockCode) FROM similar_items")
if cur.fetchone()[0] == len(uniqueItems):
    cur.execute("INSERT INTO import_status VALUES(?,?,?)", (str(datetime.now()), "similar_items","success"))
else:
    cur.execute("INSERT INTO import_status VALUES(?,?,?)", (str(datetime.now()), "similar_items","fail"))

con.commit()

In [246]:
con.close()

### Conclusion

By storing the recommendations for users and items in database beforehand, we can access recommendations faster upon load. Also, we provided an alternative to using user specific recommendations by providing similar items, which could be used for new users where we have little information on them.